<a href="https://www.kaggle.com/code/madeeltariq/api-gateway-rag?scriptVersionId=214646487" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# *Installing the required libraries*# 

In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("transformers")
install_package("pinecone-client")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")


# Imports in here

## **Code cell for Entering the ngrok API key For Ngrok implementation**

In [ ]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")

# Function to create/update .env file with Pinecone credentials
def set_pinecone_credentials():
    # Ask user for the Pinecone API key without displaying the input
    pinecone_api_key = getpass.getpass("Please enter your Pinecone API key: ")

    # Ask user for the Pinecone environment, with a default value
    pinecone_env = input("Please enter your Pinecone environment (e.g., us-west1-gcp) [default: us-east-1]: ") or "us-east-1"
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the keys; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    
    print(f"Pinecone credentials added to {env_file}")

# Call the functions to set the API key and credentials
set_ngrok_api_key()
set_pinecone_credentials()


In [ ]:
import os
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from pinecone import Pinecone

# Function to get vector from Pinecone database based on ID
def get_vector_from_pinecone(file_id, index_name="vecotr", namespace_name="newCheck", pinecone_env="us-east-1"):
    # Load environment variables from .env file
    load_dotenv()  # This will load the variables from the .env file into the environment

    # Load Pinecone API key from the environment variables
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        print("Check if ENV variables are set")

    # Ensure the API key is not None
    if not pinecone_api_key:
        # print("Error: Pinecone API key is missing from the environment variables.")
        return None

    # Initialize Pinecone client
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        print(f"Pinecone client initialized with environment: {pinecone_env}")
    except Exception as e:
        print(f"Error initializing Pinecone client: {str(e)}")
        return None

    # Initialize the index
    try:
        index = pc.Index(index_name)
    except Exception as e:
        print(f"Error initializing Pinecone index: {str(e)}")
        return None

    # Query Pinecone to retrieve vector for the specific ID
    try:
        print(f"Retrieving vector for file ID: {file_id} from Pinecone...")

        # Querying Pinecone index for the specific ID using metadata filtering
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            id=file_id,
            top_k=1,
            include_values=True,
            include_metadata=True
        )

        # Check if result contains matches
        if response and response.get("matches"):
            # Display the first few vectors based on top_k
            for idx, vector_data in enumerate(response["matches"]):
                # print(f"Vector {idx + 1} ID: {vector_data['id']}")
                print(f"Pinecone Index: {index_name}")
                print(f"Pinecone Env: {pinecone_env}")
                # print(f"Metadata (fileId): {vector_data['metadata']['fileId']}")
                # print(f"Metadata (text): {vector_data['metadata']['text'][:100]}...")  # Showing first 100 chars of text
                print(f"Vector Values: {vector_data['values'][:5]}...")  # Showing first 5 values of the vector
                print(f"Size of Vector Values: {len(vector_data['values'])}")  # Printing the size of the vector values
            return response["matches"]

        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving vector for file ID {file_id}: {str(e)}")
        return None

# Example usage of the function
file_id = "676594520c3eb8c3272efa2c"  # Replace with the file ID you want to query

vector = get_vector_from_pinecone(file_id)



# To test this one is 

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os
from pinecone import Pinecone

# Load the environment variables from .env file
load_dotenv()




# Function to retrieve and compare query vector with Pinecone vectors using cosine similarity
def find_similar_vectors(query):
    # Initialize the sentence transformer model that matches Pinecone's vector dimensionality (1024 in this case)
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can change this to a 1024-dimensional model
    
    # Vectorize the query
    query_vector = model.encode([query])[0]  # Convert the query into a vector

    # Fetch vector data from Pinecone for a sample file
    file_id = "676594520c3eb8c3272efa2c"  # Replace with your actual file ID
    result = get_vector_from_pinecone(file_id)

    if result is None:
        print("No results found.")
        return

    # Extract the first match from the result
    if isinstance(result, list) and len(result) > 0:
        match = result[0]  # Get the first match from the list of results

        # Extract the vector and metadata from the match
        stored_vector = match['values']
        metadata = match['metadata']

        # Check if the query vector and the stored vector have the same dimensionality
        if len(query_vector) != len(stored_vector):
            print(f"Error: Dimensionality mismatch! Query vector length: {len(query_vector)}, Stored vector length: {len(stored_vector)}")
            return

        # Calculate cosine similarity between the query vector and the Pinecone vector
        similarity = cosine_similarity([query_vector], [stored_vector])[0][0]

        # Print the similarity and the metadata of the most similar vector
        print(f"Cosine Similarity: {similarity:.4f}")
        print(f"Metadata (Text): {metadata.get('text', 'No text metadata available')}")
        print(f"Stored Vector: {stored_vector[:5]}...")  # Display first 5 values of the stored vector

        return similarity, metadata
    else:
        print("No valid matches found in the result.")
        return None

# Example usage
query = "What is the university name that is teaching the compiler construction course?"  # Example query
similarity, metadata = find_similar_vectors(query)


In [ ]:
from transformers import pipeline

def get_response_from_llm(query, model_name="EleutherAI/gpt-neo-1.3B"):
    """
    This function takes a user query, processes it using a free LLM from Hugging Face, 
    and returns the generated response.

    :param query: User input query.
    :param model_name: Model name to be used for text generation.
    :return: LLM's response to the query.
    """
    # Load the model pipeline
    generator = pipeline("text-generation", model=model_name, device=-1)  # Use CPU (-1) or GPU (set device to appropriate value)

    # Generate response
    response = generator(query, max_new_tokens=100, do_sample=True)
    
    # Return the generated response
    return response[0]["generated_text"]

# Example usage
user_query = "What is the significance of artificial intelligence?"
response = get_response_from_llm(user_query)
print("Response from the LLM:", response)


# Implemented Ngrok and CORS on top of the Flask API

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from pinecone import Pinecone

# Load environment variables from .env file
load_dotenv()

# Function to get vector from Pinecone database based on query and file ID
def get_vector_from_pinecone(file_id, query, index_name="vecotr", namespace_name="newCheck", pinecone_env="us-east-1"):
    # Load Pinecone API key from the environment variables
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        print("Check if ENV variables are set")
        return None

    # Initialize Pinecone client
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        index = pc.Index(index_name)
    except Exception as e:
        print(f"Error initializing Pinecone client or index: {str(e)}")
        return None

    # Embedding the query
    try:
        embedding_response = pc.inference.embed(
            model="multilingual-e5-large",
            inputs=[query],
            parameters={
                "input_type": "query",
                "truncate": "END"
            }
        )
        if embedding_response and hasattr(embedding_response, 'data') and len(embedding_response.data) > 0:
            query_embedding = embedding_response.data[0].get('values', [])
            if not query_embedding:
                print("Error: 'values' key is missing or empty in the embedding response.")
                return None
        else:
            print("Error generating embedding for the query. No data found.")
            return None
    except Exception as e:
        print(f"Error generating embedding: {str(e)}")
        return None

    # Query Pinecone
    try:
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            vector=query_embedding,
            top_k=5,
            include_values=True,
            include_metadata=True
        )
        if response and "matches" in response:
            matches = response["matches"]
            threshold = 0.8
            filtered_matches = [m for m in matches if m.get("score", 0) >= threshold]
            if not filtered_matches:
                print("No matches above the similarity threshold.")
                return None
            filtered_matches = sorted(filtered_matches, key=lambda x: x.get('score', 0), reverse=True)
            return filtered_matches
        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving vector for file ID {file_id}: {str(e)}")
        return None

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/get_model_response', methods=['POST'])
def get_model_response():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "Request body must be JSON"}), 400
        file_id = data.get("file_id")
        query = data.get("query")
        if not file_id:
            return jsonify({"error": "'file_id' is required"}), 400
        vector_response = get_vector_from_pinecone(file_id, query)
        
        if vector_response:
            enriched_matches = []
            for vector_data in vector_response:
                # Ensure vector_data is safely converted into a dictionary-like structure
                if hasattr(vector_data, "to_dict"):
                    vector_data = vector_data.to_dict()
        
                # Filter out unnecessary data and include similarity score
                
                filtered_data = {k: v for k, v in vector_data.items() if k != 'values'}
                # filtered_data['similarity'] = vector_data.get('score', None)
        
                enriched_matches.append(filtered_data)
        
            return jsonify({"status": "success", "data": enriched_matches}), 200
        else:
            return jsonify({"status": "error", "message": "No vector found for the given file_id"}), 404

    except Exception as e:
        print(f"[ERROR] Exception occurred: {str(e)}")
        return jsonify({"status": "error", "message": "An error occurred"}), 500

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

def run_flask():
    app.run(host="0.0.0.0", port=5000)

def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()
public_url = run_ngrok()

while True:
    time.sleep(1)


# *Working Prototype for flask listening***

In [ ]:
import os
from flask import Flask, request, jsonify
import threading
import time

# Initialize Flask app
app = Flask(__name__)

# Health check endpoint
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "Flask server is running Greate!"})

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 8000...")
    app.run(host="0.0.0.0", port=9000)  # Running on port 8000

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Print out the URL where Flask is running
print("Flask server should be accessible at: http://0.0.0.0:8000")

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


# How to send request from the JavaScript Server Side 